# Cutting stock problem


In [1]:
%load_ext gams.magic

Model below finds patterns explicitly (with repeats)

In [2]:
%%gams
set i widths / w1*w3 /;
set j / 1*16 /;
parameter w(i) width / w1 4, w2 5, w3 6 /,
          dem(i) / w1 12, w2 15, w3 22 /,
          bnd(i);
scalar stocklen /19/;
bnd(i) = floor(stocklen/w(i));

integer variables x(i,j) 'number widths in pattern';
binary variables z(j) 'use pattern';
variables obj;

equations fixed(i,j), demand(i), defobj, cuts(j), symmetry(j);

fixed(i,j)..
  x(i,j) =l= w(i)*z(j);

demand(i)..
  sum(j, x(i,j)) =g= dem(i);

defobj..
  obj =e= sum(j, z(j));

cuts(j)..
  sum(i, w(i)*x(i,j)) =l= stocklen;

symmetry(j)$(j.ord lt card(j))..
  z(j) =g= z(j+1);

model cutting /all/;
cutting.optcr = 0;
solve cutting using mip min obj;

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),14.0,83,65,MIP,CPLEX,0.018


In [3]:
m = gams.exchange_container
x = m.data['x']
z = m.data['z'].records.drop(['marginal','lower','upper','scale'],axis=1)
display(x.pivot(),z[z['level'] > 0])
gams.reset()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
w1,1.0,1.0,1.0,0.0,0.0,2.0,2.0,0.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0
w2,3.0,1.0,3.0,1.0,0.0,1.0,1.0,0.0,1.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0
w3,0.0,1.0,0.0,2.0,3.0,1.0,1.0,3.0,1.0,0.0,1.0,3.0,3.0,3.0,0.0,0.0


,j,level
0,1,1.0
1,2,1.0
2,3,1.0
3,4,1.0
4,5,1.0
5,6,1.0
6,7,1.0
7,8,1.0
8,9,1.0
9,10,1.0


Model below selects patterns from a predefined set (of columns)

In [4]:
%%gams
set i widths / w1*w3 /;
set p patterns / 1*9 /;
parameter dem(i) / w1 12, w2 15, w3 22 /;
integer variable y(p);
variable obj;
table A(i,p) 
   1 2 3 4 5 6 7 8 9
w1 0 0 1 0 2 1 2 3 4
w2 0 1 0 2 1 3 2 1 0
w3 3 2 2 1 1 0 0 0 0;

equations cover(i), defobj2;

cover(i)..
  sum(p, A(i,p)*y(p)) =g= dem(i);

defobj2..
  obj =e= sum(p, y(p));

model cutpatt /defobj2, cover/;
y.up(p) = 1e5;
cutpatt.optcr = 0;
solve cutpatt using mip min obj;

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),14.0,4,10,MIP,CPLEX,0.004


In [5]:
m = gams.exchange_container
y = m.data['y'].records.drop(['marginal','lower','upper','scale'],axis=1)
display(y[y['level'] > 0])
%gams_reset

,p,level
0,1,6.0
4,5,4.0
5,6,4.0


Model below implements column generation for this problem

In [6]:
%%gams
* This is the data from lecture 21 notes
Set  i widths    /w1*w3/;
Parameter
     stocklen raw width /19/,
     w(i) width     /w1  4
                     w2  5
                     w3  6/,
     dem(i) demand  /w1 12
                     w2 15
                     w3 22/;

In [7]:
%%gams
* $include cutstock3.gms

* Gilmore-Gomory column generation algorithm

Set  p        possible patteqrns  /p1*p1000/
     pp(p)    dynamic subset of p;
Parameter
     A(i,p) number of width i in pattern growing in p;

* Master model
Variable x(p)     patterns used
         z         objective variable;
Integer variable x; x.up(p) = sum(i, dem(i));

Equation numpat    number of patterns used
         demand(i) meet demand;

numpat..     z =e= sum(pp, x(pp));
demand(i)..  sum(pp, A(i,pp)*x(pp)) =g= dem(i);

model master /numpat, demand/;

* Pricing problem - Knapsack model
Variable  y(i) new pattern;
Integer variable y; y.up(i) = ceil(stocklen/w(i));

Equation defobj
         knapsack knapsack constraint;

defobj..     z =e= sum(i, demand.m(i)*y(i));
knapsack..   sum(i, w(i)*y(i)) =l= stocklen;

model pricing /defobj, knapsack/;

In [8]:
%%gams
* Initialization - the initial patterns have a single width
* could do this in python
pp(p) = ord(p)<=card(i);
A(i,pp(p))$(ord(i)=ord(p)) = floor(stocklen/w(i));
display A;

Scalar done  loop indicator /0/
Set    pi(p) set of the last pattern; 
pi(p) = yes$(ord(p)=card(pp)+1);

In [9]:
%%gams
option optcr=0,limrow=0,limcol=0,solprint=off;

While(not done and card(pp)<card(p),
   solve master using rmip minimizing z;
   solve pricing using mip maximizing z;

* pattern that might improve the master model found?
* 1 is cost of adding that pattern, z.l is resulting decrease
   if(1 + 0.001 < z.l,
      A(i,pi) = round(y.l(i));
      pp(pi) = yes; pi(p) = pi(p-1);
   else
      done = 1;
   );
);
display 'lower bound for number of rolls', master.objval;

solve master using mip minimizing z;

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),15.3333,4,4,RMIP,CPLEX,0.001
1,Normal (1),Optimal Global (1),1.2500,2,4,MIP,CPLEX,0.003
2,Normal (1),Optimal Global (1),14.0833,4,5,RMIP,CPLEX,0.001
3,Normal (1),Optimal Global (1),1.0833,2,4,MIP,CPLEX,0.003
4,Normal (1),Optimal Global (1),13.8889,4,6,RMIP,CPLEX,0.001
5,Normal (1),Optimal Global (1),1.0370,2,4,MIP,CPLEX,0.004
6,Normal (1),Optimal Global (1),13.7333,4,7,RMIP,CPLEX,0.001
7,Normal (1),Optimal Global (1),1.0000,2,4,MIP,CPLEX,0.003
8,Normal (1),Optimal Global (1),14.0000,4,7,MIP,CPLEX,0.003


In [10]:
m = gams.exchange_container
x = m.data['x'].records.drop(['marginal','lower','upper','scale'],axis=1)
A = m.data['A']
display(x[x['level'] > 0],A.pivot())

,p,level
2,p3,6.0
3,p4,4.0
5,p6,4.0


,p1,p4,p5,p6,p2,p3
w1,4.0,1.0,3.0,2.0,0.0,0.0
w2,0.0,3.0,0.0,1.0,3.0,0.0
w3,0.0,0.0,1.0,1.0,0.0,3.0


In [11]:
%gams_cleanup --closedown